To aid in generating an equity research report for Bendigo and Adelaide bank, first a fundimental analysis is performed by defining relevent functions to calculate key profitability, liquidity and efficiency and valuation ratios. 

In [ ]:
def net_interest_margin(net_interest, earning_assets):
    nim = net_interest / earning_assets
    return nim
def calculate_dividend_yield(dividend, last_price):
    if last_price == 0:
        return 0
    return dividend / last_price
def calculate_pe_ratio(last_price, eps):
    if eps == 0:
        return 0
    return last_price / eps
def calculate_eps(earnings, shares_outstanding):
    if shares_outstanding == 0:
        return 0
    return earnings / shares_outstanding
def calculate_roa(earnings, total_assets):
    if total_assets == 0:
        return 0
    return earnings / total_assets
def calculate_pe_ratio(last_price, eps):
    if eps == 0:
        return 0
    return last_price / eps
def calculate_eps(earnings, shares_outstanding):
    if shares_outstanding == 0:
        return 0
    return earnings / shares_outstanding

To provide 

Specific 

Comentary for the company was developed from all previous analysis to include an executive summary, business overview, investment theses and analysis of key finanical ratios. Calculated financial data was inserted into the text using the "in" python function. 

In [ ]:
with open("commentary.txt", "r") as file:
    lines = file.read()

from fpdf import FPDF

# Create a PDF object
my_pdf = FPDF(orientation='P', unit='mm', format='A4')

# Add a page
my_pdf.add_page()

# Set font
my_pdf.set_font('Times','', 11)

# Set automatic page break
my_pdf.set_auto_page_break(auto=True, margin=15)

# Add text
my_pdf.multi_cell(0, 10, lines)

# Save the PDF
my_pdf.output('hello_world.pdf')




RuntimeError: FPDF error: Undefined font: times new roman B